In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score, recall_score, precision_score
forest = RandomForestRegressor(n_estimators=1000,criterion='squared_error',random_state=1,n_jobs=-1)
gdp = pd.read_csv("../matlab文件/处理后数据/人均GDP.csv").values[1:,2:]
gdp_inc = pd.read_csv("../matlab文件/处理后数据/人均GDP增速.csv").values[1:,2:]
out_of_work = pd.read_csv("../matlab文件/处理后数据/失业率.csv").values[1:,2:]
cpi = pd.read_csv("../matlab文件/处理后数据/CPI.csv").values[:,4:]
college_rate = pd.read_csv("../matlab文件/处理后数据/入学率.csv").values[1:,3:]
inflation = pd.read_csv("../matlab文件/处理后数据/通货膨胀.csv").values[:,3:30]
conflict = pd.read_csv("../matlab文件/处理后数据/冲突.csv").values[:,1:]
Country = ['Armenia','Azerbaijan','Barbados','Belarus','Ecuador','Georgia','Germany','Guinea','Japan','Kazakhstan','Kuwait','Kyrgyzstan','Lithuania','Russia','Tajikistan','Tanzania','Tunisia','Turkey','Turkmenistan'] 
data = np.zeros((21*27,7),float)
for i in range(21):
    conflict[i] = (conflict[i] * 100)/(sum(conflict[i]))
data[:,0] = gdp.reshape(21*27)
data[:,1] = gdp_inc.reshape(21*27)
data[:,2] = out_of_work.reshape(21*27)
data[:,3] = cpi.reshape(21*27)
data[:,4] = college_rate.reshape(21*27)
data[:,5] = inflation.reshape(21*27)
data[:,6] = conflict.reshape(21*27)
y = np.zeros((21,27),bool)
y[0][10] = 1
y[1][10] = 1
y[3][10] = 1
y[5][8] = 1
y[9][10] = 1
y[11][10] = 1
y[13][10] = 1
y[14][11] = 1
y[18][10] = 1
y[19][9] = 1
y[20][10] = 1
y = y.reshape(567)
x_train,x_test,y_train,y_test = train_test_split(data,y,test_size =0.84,random_state = 211)
forest.fit(x_train, y_train)
y_pre = forest.predict(x_test).astype(bool)
f1 = f1_score(y_test, y_pre)
accuracy = accuracy_score(y_test,y_pre)
recall = recall_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
print("\naccuracy_score = {}\nrecall_score = {}\n".format(accuracy,recall))
importances = list(forest.feature_importances_)
print(importances)


accuracy_score = 0.7840670859538784
recall_score = 0.7777777777777778

[0.016133870975143155, 0.27124328336250164, 0.01683815759318027, 0.024425250495824176, 0.013710197760681922, 0.1388210119528406, 0.5188282278598283]


In [29]:
import numpy as np
import pandas as pd
w = [0.016133870975143155, 0.27124328336250164, 0.01683815759318027, 0.024425250495824176, 0.013710197760681922, 0.1388210119528406, 0.5188282278598283]
gdp = pd.read_csv("../matlab文件/处理后数据/人均GDP.csv").values[1:,2:]
gdp_inc = pd.read_csv("../matlab文件/处理后数据/优化后人均gdp增速.csv").values[:,1:]
out_of_work = pd.read_csv("../matlab文件/处理后数据/失业率.csv").values[1:,2:]
cpi = pd.read_csv("../matlab文件/处理后数据/CPI.csv").values[:,4:]
college_rate = pd.read_csv("../matlab文件/处理后数据/入学率.csv").values[1:,3:]
inflation = pd.read_csv("../matlab文件/处理后数据/通货膨胀.csv").values[:,3:30]
conflict = pd.read_csv("../matlab文件/处理后数据/冲突.csv").values[:,1:]
Country = ['Armenia','Azerbaijan','Barbados','Belarus','Ecuador','Georgia','Germany','Guinea','Japan','Kazakhstan','Kuwait','Kyrgyzstan','Lithuania','Russia','Tajikistan','Tanzania','Tunisia','Turkey','Turkmenistan'] 
score = np.zeros((21*27),float)
data = np.zeros((21*27,7),float)
w_ma = np.zeros((21*27,7),float)
coefficient = np.zeros((7,),float)
for i in range(567):
    w_ma[i] = w
for i in range(21):
    conflict[i] = (conflict[i])/(sum(conflict[i]))
ideal = [0,0,0,0,0,0,0]
worse = [0,0,0,0,0,0,0]
data[:,0] = gdp.reshape(21*27)
data[:,1] = gdp_inc.reshape(21*27)
data[:,2] = out_of_work.reshape(21*27)
data[:,3] = cpi.reshape(21*27)
data[:,4] = college_rate.reshape(21*27)
data[:,5] = inflation.reshape(21*27)
data[:,6] = conflict.reshape(21*27)
for j in range(7):
    add = 0
    for i in range(567):
        data[i][j] = float(data[i][j])
        add += pow(data[i][j],2)
    add = pow(add,0.5)
    coefficient[j] = add
    data[:,j] = data[:,j]/add
data = data*w_ma
for j in [0,1,3,4]:
    max_value = np.max(data[:,j])
    min_value = np.min(data[:,j])
    ideal[j] = max_value
    worse[j] = min_value
for j in [2,5,6]:
    max_value = np.max(data[:,j])
    min_value = np.min(data[:,j])
    ideal[j] = min_value
    worse[j] = max_value
for i in range(567):
    l1 = 0
    l2 = 0
    for j in range(7):
        l1 += pow((ideal[j] - data[i][j]),2)
        l2 += pow((worse[j] - data[i][j]),2)
    l1 = pow(l1,0.5)
    l2 = pow(l2,0.5)
    score[i] = l2/(l1+l2)
score = score.reshape(21,27)
score = pd.DataFrame(score)
score.to_csv("../matlab文件/处理后数据/result_2.csv")